# 03 Anomaly Detection

Detect macro anomalies and interpret crisis periods.


## Table of Contents
- [Fit detector](#fit-detector)
- [Inspect anomalies](#inspect-anomalies)
- [Compare to recessions](#compare-to-recessions)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Unsupervised notebooks help you understand macro structure:
- latent factors (PCA),
- regimes (clustering),
- anomalies/crises.


## Prerequisites (Quick Self-Check)
- Completed Part 01 (macro panel) or equivalent.
- Comfort with standardization and basic linear algebra intuition (variance, distance).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Forgetting to standardize features before PCA/clustering.
- Over-interpreting clusters as causal regimes.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/04_unsupervised/03_anomaly_detection.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/04_unsupervised/03_anomaly_detection.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Detect unusual macro periods (potential crises) using anomaly detection.

This is not about predicting recessions directly. It's about:
- flagging periods that look "different" in feature space
- interpreting what those periods correspond to historically



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


<a id="fit-detector"></a>
## Fit detector

### Goal
Fit an anomaly detector on standardized macro features.



### Your Turn (1): Load data and choose features


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

x_cols = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']
df_m = df[x_cols + ['recession']].dropna().copy()
df_m.head()



### Your Turn (2): Standardize and fit IsolationForest


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

sc = StandardScaler().fit(df_m[x_cols])
X = sc.transform(df_m[x_cols])

# TODO: Choose contamination (expected fraction of anomalies)
iso = IsolationForest(contamination=0.05, random_state=0)
iso.fit(X)

# Higher score means more normal in sklearn; we'll invert for "anomaly score"
score_normal = iso.score_samples(X)
df_m['anomaly_score'] = -score_normal
df_m[['anomaly_score']].head()



<a id="inspect-anomalies"></a>
## Inspect anomalies

### Goal
Identify the most anomalous periods and inspect them.



### Your Turn (1): List top anomalous dates


In [ ]:
# TODO: Sort by anomaly_score and print the top 10 most anomalous dates.
df_m[['anomaly_score']].sort_values('anomaly_score', ascending=False).head(10)



### Your Turn (2): Plot anomaly score over time


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot df_m['anomaly_score'] over time.
...



<a id="compare-to-recessions"></a>
## Compare to recessions

### Goal
Compare anomaly flags to technical recession labels.

Questions:
- Do anomalies cluster around recessions?
- Are there anomalies outside recessions (e.g., inflation shocks)?



### Your Turn: Simple comparison


In [ ]:
# TODO: Define a binary anomaly flag (top X%) and compute recession rate among anomalies.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Confirm your feature matrix is standardized (or justify why not).
# Example:
# assert abs(X_scaled.mean(axis=0)).max() < 1e-6
# assert abs(X_scaled.std(axis=0) - 1).max() < 1e-6
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Fit detector</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_anomaly_detection — Fit detector
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()
X = StandardScaler().fit_transform(panel)
iso = IsolationForest(contamination=0.05, random_state=0).fit(X)
score = -iso.decision_function(X)
anomaly = pd.Series(score, index=panel.index, name='anomaly_score')
anomaly.sort_values(ascending=False).head(10)
```

</details>

<details><summary>Solution: Inspect anomalies</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_anomaly_detection — Inspect anomalies
# Plot anomaly_score over time and inspect peaks.
```

</details>

<details><summary>Solution: Compare to recessions</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_anomaly_detection — Compare to recessions
# Compare anomaly peaks to recession labels.
```

</details>

